In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import math
import matplotlib.patches as mpatches

dataFolder = "D:\\Uni\\DataOutput\\"
fName_2MIN = "TerminusEst_MCTS_2MIN"

In [1]:
def remove_space_from_column_names(df):
    for col in df.columns:
        if (col[0] == " "):
            df = df.rename(columns={col: col[1:]})
    
    return df

Gathering the data from the quick run file.

In [2]:
l_post = []
for i in (range(1, 50, 10)):
    l_post.append((i, i+9))

qRunFiles = []
for post in l_post:
    qRunFiles.append(dataFolder + fName_2MIN + "_" + str(post[0]) + "_" + str(post[1]) + ".csv")

dataQR = pd.read_csv(qRunFiles[0])

for d in range (1, 5):
    dataQR = pd.concat([dataQR, pd.read_csv(qRunFiles[d])], axis=0)

# Appears to be some error in column naming where a single space is in front...


for col in dataQR.columns:
    if (col[0] == " "):
        dataQR = dataQR.rename(columns={col: col[1:]})

time_searched = dataQR.TIME_TOTAL - dataQR.TIME_BUILD_SEARCH_TREE
dataQR.insert(3, "TIME_SEARCH", time_searched, True)
dataQR = dataQR.sort_values(by=['ID']).reset_index(drop=True)

NameError: name 'dataFolder' is not defined

In [ ]:
# Quick look at the data.
dataQR

We separate the data based on whether the hybridization number was found.

In [ ]:
d_known = dataQR.loc[dataQR.HYB_EXACT >= 0]
d_unknown = dataQR.loc[dataQR.HYB_EXACT < 0]

print("Finished instances: ", len(d_known), "\tUnfinished instances: ", len(d_unknown))

In [ ]:
sns.scatterplot(d_known.HYB_EXACT, d_known.TIME_SEARCH, alpha=0.5)

Divide into datasets of different difficulty.

In [ ]:
# Defining the datasets.
d_easy = d_known.loc[d_known.TIME_SEARCH <= 10]
d_medium = d_known.loc[d_known.TIME_SEARCH > 10]
d_hard = d_unknown

In [ ]:
print("Sizes.\nEasy: ", len(d_easy), "\tMedium: ", len(d_medium), "\tHard: ", len(d_hard))

In [ ]:
# Separating the datasets into their own separate text files.
def separate_datasets():
    location = "D:\\Uni\\DataInput\\"

    def save_file_names(d, name, location):
        fName = location+name
        f = open(fName, "w+")

        for point in d:
            f.write(point + "\n")

        f.close()

    save_file_names(d_easy.ID.values, "data_easy.txt", location)
    save_file_names(d_medium.ID.values, "data_medium.txt", location)
    save_file_names(d_hard.ID.values, "data_hard.txt", location)

Getting the data from the run with basic TerminusEst (10min runtime)

In [ ]:
def collect_files_with_text(text, path):
    files = []
    for r, d, f in os.walk(path):
        for file in f:
            if text in file:
                files.append(os.path.join(r, file))
    
    return files

def get_one_dataframe(files):
    d = pd.read_csv(files[0])
    for i in range(1, len(files)):
        d = pd.concat([d, pd.read_csv(files[i])], axis=0, sort=False)
    return d

filesB = collect_files_with_text('basicTE', dataFolder)
dataB = get_one_dataframe(filesB)
dataB = remove_space_from_column_names(dataB)
dataB

In java, I had to use -1 to mark data which was not registered. Correcting for this.

In [ ]:
def set_lower_bound(x):
    if (x.BASIC_HYB == -1 and math.isnan(x.BASIC_LB)):
        print("ERROR! LB NOT LOGGED.")
    elif (not math.isnan(x.BASIC_LB)):
        if (x.BASIC_LB <= 0):
            print("ERROR!")
            print(x)
            return -1
        else:
            return x.BASIC_LB
    else:
        return x.BASIC_HYB
    
dataB["LB_2"] = dataB.apply(set_lower_bound, axis=1)
dataB = dataB.sort_values(by=['ID']).reset_index(drop=True)

In [ ]:
def row_match(d1, d2, row_id):
    if (len(d1.index) != len(d2.index)):
        return false
    
    matches = d1[row_id] != d2[row_id]
    return not matches.any()

row_match(dataB, dataQR, 'ID')

Display the bounds for the data.

In [ ]:
lb = dataB[['ID', 'LB_2']]
ub = dataQR[['ID', 'HYB_UPPER', 'TIME_BUILD_SEARCH_TREE']]
bounds = pd.merge(lb, ub, on='ID')
bounds = bounds.rename(columns={'LB_2':'LB', 'HYB_UPPER':'UB'})
bounds['AR'] = bounds.UB / bounds.LB
bounds['EXACT'] = bounds.LB == bounds.UB

In [ ]:
bounds['DIFF'] = bounds.UB - bounds.LB
bounds

In [ ]:
bounds.mean()

In [ ]:
id_easy = d_easy.ID.values
id_medium = d_medium.ID.values
id_hard = d_hard.ID.values

Get bounds data by difficulty

In [ ]:
bounds_easy = bounds[bounds.ID.isin(id_easy)]
bounds_medium = bounds[bounds.ID.isin(id_medium)]
bounds_hard = bounds[bounds.ID.isin(id_hard)]

In [ ]:
print("Easy dataset AAS: \n", bounds_easy.mean(),"\n\n", bounds_easy.std())
print("\nMedium dataset AAS: \n", bounds_medium.mean(), "\n\n", bounds_medium.std())
print("\nHard dataset AAS: \n", bounds_hard.mean(), "\n\n", bounds_hard.std())

In [ ]:
def nonexact(d):
    temp = d[(d.UB-d.LB) >=1]
    return (temp.UB - temp.LB)
    

sns.set(style='white')
fig, ax =plt.subplots()
ax = sns.distplot(nonexact(bounds_hard), kde=False, bins=5, axlabel="Difference")
ax.set_xticks([1,2,3,4,5])
# ax.set_xticklabels([1,2,3,4,5])
# ax.bar([1,2,3,4,5], 3, align='center')

Presenting parameter tuning info.

In [ ]:
def remove_incomplete(d):
    return d.loc[d.UPPER_BOUND != -1]

def unknown_to_20(x):
    if x.UPPER_BOUND == -1:
        return 20
    else:
        return x.UPPER_BOUND

def worst_case_incomplete(d):
    return d.apply(lambda x: unknown_to_20(x), axis=1)

# Parameter tuning data presentation.
def present_param_tuning_info(name):
    f = collect_files_with_text(name, dataFolder)
    d = get_one_dataframe(f)
    d = remove_space_from_column_names(d)
    # d.UPPER_BOUND = worst_case_incomplete(d)
    # d.DEPTH_FIRST_SOLUTION = worst_case_incomplete(d)
    
    # Get the lower bound for the relevant entries
    lbs = bounds[bounds.ID.isin(d.ID)]
    temp = pd.merge(d, lbs[['ID', 'LB']], on='ID')
    d['AAR'] = temp.UPPER_BOUND / temp.LB
    d['DIFF'] = temp.UPPER_BOUND - temp.LB
    d['DIFF_FIRST'] = temp.DEPTH_FIRST_SOLUTION - temp.UPPER_BOUND
    
    return d
    

pt_hard = present_param_tuning_info("treeTE_hard_2_2")
pt_hard_c = remove_incomplete(pt_hard)
subset = pt_hard[['AAR', 'DIFF', 'DIFF_FIRST', 'DEPTH_AVG', 'SHALLOWEST_LEAF', 'DEEPEST_NODE']]
print(subset.mean())
print("\nIncomplete: ", len(pt_hard_c.index) / len(pt_hard.index))
print(subset.std())

Presenting meta search info.

In [ ]:
def present_meta_search_info(name):
    f = collect_files_with_text(name, dataFolder)
    
    points = []
    for file in f:
        d = pd.read_csv(file)
        lbs = bounds[bounds.ID.isin(d.ID)]
        temp = pd.merge(d, lbs[['ID', 'LB']], on='ID')
        temp['DIFF'] = temp.UPPER_BOUND - temp.LB
        if not 'T_COMPLETED' in temp.columns:
            temp['T_COMPLETED'] = temp.trees[0]
        points.append([temp.trees[0], temp.DIFF.mean(), (temp.trees > temp.T_COMPLETED).any()])
    
    return (points)


l_meta1 = present_meta_search_info("metaTE_50k_hard")
l_meta1.sort()

l_meta2 = present_meta_search_info("metaTE_hard")
l_meta2.sort()

In [ ]:
d_meta1 = pd.DataFrame(l_meta1)
d_meta2 = pd.DataFrame(l_meta2)

In [ ]:

d_meta1

In [ ]:
plt.plot(d_meta1[0], d_meta1[1])
plt.plot(d_meta2[0], d_meta2[1])
plt.ylabel('Average Diff.')
plt.xlabel('Number of runs')

yellow = mpatches.Patch(color='orange', label="10'000 nodes")
blue = mpatches.Patch(color='tab:blue', label="50'000 nodes")
plt.legend(handles=[yellow, blue])

Getting information about hard dataset when run with TerminusEstMCTS

In [ ]:
db_hard = dataB[dataB.ID.isin(id_hard)]
print("Avg. Diff: ", db_hard[db_hard.BASIC_HYB >= 0].BASIC_HYB.mean())
print("Percent found: ", len(db_hard[db_hard.BASIC_HYB >= 0].index) / len(db_hard.index))
print("AVg. time: ", dataB.BASIC_RUNTIME.mean())

db_hard

In [ ]:
f_tem = collect_files_with_text("TE_MCTS", dataFolder)
f_tem

In [ ]:
d_tem = get_one_dataframe(f_tem)
d_tem = remove_space_from_column_names(d_tem)
d_tem

In [ ]:
d_t = d_tem[['ID','TIME_TOTAL', 'CANCELED']].rename(columns={'TIME_TOTAL':'TIME_TEMCTS', 'CANCELED':'CANCELED_TEMCTS'})
d_b = dataB[dataB.ID.isin(d_t.ID)][['ID', 'BASIC_RUNTIME', 'BASIC_HYB', 'LB_2']]

In [ ]:
dataB.head()

In [ ]:
d_comp = pd.merge(d_t, d_b, on="ID")

Compare the TerminusEst and TerminusEstMCTS on the hard dataset.

In [ ]:
def thing_1(x):
    if x.CANCELED_TEMCTS == ' false':
        return 1
    else:
        return 0

def thing_2(x):
    if x.BASIC_CANCELED:
        return 0
    else:
        return 1

d_comp['BASIC_CANCELED'] = d_comp.BASIC_HYB == -1
d_comp['TEMCTS_S'] = d_comp.apply(thing_1, axis=1)
d_comp['BASIC_S'] = d_comp.apply(thing_2, axis=1)
d_comp.head()

In [ ]:
d_comp[['TIME_TEMCTS', 'BASIC_RUNTIME', 'TEMCTS_S', 'BASIC_S']].mean()

Plotting difference across LB found.

In [ ]:
l = []
bounds.head()
for name, group in bounds.groupby('UB'):
    l.append([name, (group.EXACT.sum()/len(group.index)), len(group.index)])
    
pd.DataFrame(l)

In [ ]:
bounds[bounds.UB - bounds.LB == 1]